In [4]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from pydantic import BaseModel
from typing import Literal
from fastapi import FastAPI, UploadFile, File, HTTPException, Depends

In [6]:
app =FastAPI(title="1099FLOW Backend")
#define the possible states for the workflow
WorkflowState = Literal["Uploaded", "Verified (AI)", "Approved", "Submitted"]

In [7]:
pip install python-multipart

Note: you may need to restart the kernel to use updated packages.


In [8]:
# pydantic Data Models (Schemas)
class Vendor(BaseModel):
    vendor_id: str
    current_state: WorkflowState
    ein: str | None = None
    # other vendor fields
    
class Workflowlog(BaseModel):
    vendor_id: str
    timestamp: str
    agent_id: str # AI or User ID
    old_state: WorkflowState
    new_state: WorkflowState
        
# supabase Intergration (Conceptal)
# Use a service layer to interact with Supabase via PostgRest/Postgres client
def get_supabase_clienr():
    #initialize and return Supabase client/connection pool
    pass

# Endpoints
@app.post("/document/upload/{vendor_id}")
async def upload_document(vendor_id: str, file: UploadFile = File(...)):
    # 1. Store file in Object Storage( Supabase Storage or DigitalOcean Spaces).
    # 2. Trigger OCR (see Section 3).
    # 3. Call state machine to log transitions: upload-> Verified (AI).
    return {"status": "Upload successful, verification pending "}

In [9]:
pip install google-cloud-documentai

Note: you may need to restart the kernel to use updated packages.


In [10]:
# ocr_service.py (External dependency call)
from google.cloud import documentai # Conceptual libray

def perform_ocr_and_validate(file_path: str, vendor_id: str, client):
    # 1.Call Document AI API
    extracted_data = documentai.process_document(file_path)
    
    # 2. Extract key fields
    ein = extracted_data.get("EIN")
    
    # 3. Simple validation
    if len(ein) == 9 and ein.isdigit():
        transition_state(vendor_id, "Verified (AI)", "AI_OCR_SERVICE", client)
        #update vendor table with extracted EIN, etc.
    else:
        #flag for human review
        transition_state(vendor_id, "Verified (Flagged)", "AI_OCR_SERVICE", client)

C:\Users\USER\anaconda3\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.9) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [11]:
pip install google-cloud-storage

In [12]:
pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [13]:
#pdf_generator.py
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def generate_1099_pdf(vendor_data, bookkeepper_data):
    # 1. Create a PDF canvas
    c = canvas.Canvas("1099-NEC.pdf", pagesize=letter)
    
    # 2. Draw text and data on the canvas at precise coordinates
    c.drawString(100, 700, f"payer Name: {bookkeeper_data['name']}")
    c.drawString(100, 650, f"Recipient EIN: {vendor_data['ein']}")
    
    #3. Save the PDF
    c.save()
    
    #4. Update state to 'submitted' and save PDF link to supabase
    transition_state(vendor_data['vendor_id'], "submitted", "User/System", client)

In [14]:
import requests  # or use an async client like httpx

def send_notification(recipient_email, subject, body):
    # API call to external email service
    headers = {"Authorization": "Bearer YOUR_API_KEY"}
    payload = {"to": recipient_email, "subject": subject, "html": body}
    
    response = requests.post("EMAIL_SERVICE_API_URL", headers=headers, json=payload)
    response.raise_for_status()


# Integrated into workflow_service.py:
def transition_state(vendor_id, old_state, new_state):
    """
    Updates state and sends notifications depending on the state transition.
    """
    # ... your state update logic here ...

    if new_state == "Verified (AI)":
        send_notification(
            "bookkeeper@org.com",
            "Verification Complete",
            "A vendor's W-9 is ready for human review."
        )

    elif new_state == "Submitted":
        send_notification(
            "vendor@org.com",
            "Forms Filed",
            "Your 1099-NEC has been successfully filed."
        )

        

In [12]:
# email_service.py
import requests # or use an async client like httpx

def send_notification(recipient_email, subject, body):
    # API call to external email service
    headers = {"Authorization": "Bearer YOUR_API_KEY"}
    payload = {"to": recipient_email, "subject": subject, "html": body}
    
    response = requests.post("EMAIL_SERVICE_API_URL", headers=headers, json=payload)
    response.raise_for_status()

# Integrated into workflow_service.py:
def transition_state(...):
    # ... state updates ...
    if new_state == "Verified (AI)":
        send_notification("bookkeeper@org.com", "Verification Complete", "A vendor's W-9 is ready for human review.")
    elif new_state == "Submitted":
        send_notification("vendor@org.com", "Forms Filed", "Your 1099-NEC has been successfully filed.")

SyntaxError: invalid syntax (1222215875.py, line 13)